### Exemplo interativo - controlador PID aplicado a um sistema de segunda ordem

In [1]:
# pip install control
# pip install tbcontrol
# pip install ipywidgets

In [2]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from tbcontrol.loops import feedback
import control as ct
import ipywidgets as widgets
from ipywidgets import interactive

In [3]:
s = ct.tf([1, 0], 1)
ts = np.linspace(0.001, 40, 500)

In [4]:
def plotresponse(ax, G, *args, **kwargs):
    ax.plot(*ct.step_response(G, T=ts), *args, **kwargs)

In [5]:
cp = {
    'red': (1.0, 0.349, 0.369, 1.0),
    'green': (0.541, 0.788, 0.149, 1.0),
    'blue': (0.098, 0.510, 0.769, 1.0),
    'lred': (1.0, 0.588, 0.6, 1.0),
    'lgreen': (0.722, 0.894, 0.443, 1.0),
    'lblue': (0.369, 0.706, 0.918, 1.0),
    'orange': (1.0, 0.506, 0.227, 1.0),
    'yellow': (1.0, 0.792, 0.227, 1.0),
    'pink': (1.0, 0.349, 0.611, 1.0),
    'purple': (0.416, 0.298, 0.576, 1.0),
    'turquoise': (0.098, 0.761, 0.769, 1.0),
    'brown': (0.576, 0.380, 0.298, 1.0)
    }

def update_plot(ζ, P, Ti, Td):
    Gp = 1/(s**2 + 2*ζ*s + 1)
    Gc = P*(1 + 1/(Ti*s) + (Td*s)/(1+0.001*(Td*s)))
    fig, (outputs, errors, errorint, u) = plt.subplots(
        4, 1, figsize=(5, 10), sharex=True)
    outputs.plot(ts, np.ones_like(ts), color='black')
    plotresponse(outputs, Gp, color=cp['blue'], label='Planta sem controle')
    # plotresponse(outputs, feedback(P*Gp, 1), color='teal', label='Controle P')
    plotresponse(outputs, feedback(Gc*Gp, 1), color=cp['pink'], label='Planta com controlador PID')
    outputs.set_ylabel(r'$y$')
    outputs.legend()

    # plotresponse(errors, 1 - feedback(P*Gp, 1), color='teal')
    plotresponse(errors, 1 - feedback(Gc*Gp, 1), color=cp['red'])
    errors.set_ylabel(r'$e$')

    # plotresponse(u, feedback(P, Gp), color='teal')
    plotresponse(u, feedback(Gc, Gp), color=cp['orange'])
    u.set_ylabel('$u$')
    u.set_xlabel(r'$ω_n t$')
    u.set_ylim(0,1.25)

    plotresponse(errorint, (1 - feedback(Gc*Gp, 1))/s, color=cp['brown'])
    errorint.set_ylabel(r'$∫e$')
    

In [6]:
# Create sliders
ζ_slider = widgets.FloatSlider(
    value=0.25, min=0.0, max=2.0, step=0.01, description='ζ:')
P_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=2.0, step=0.1, description='Kp:')
Ti_slider = widgets.FloatSlider(
    value=1.0, min=0.01, max=2.0, step=0.01, description='Ti:')
Td_slider = widgets.FloatSlider(
    value=1.0, min=0.0, max=2.0, step=0.01, description='Td:')

# Create the interactive plot
interactive_plot = interactive(
    update_plot, ζ=ζ_slider, P=P_slider, Ti=Ti_slider, Td=Td_slider)
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(FloatSlider(value=0.25, description='ζ:', max=2.0, step=0.01), FloatSlider(value=1.0, de…

#### Influência do controlador nos polos

In [7]:
s_ = sp.Symbol('s')
zt_ = sp.Symbol('ζ')
K_p = sp.Symbol('K_p')
K_i = sp.Symbol('K_i')
K_d = sp.Symbol('K_d')

In [8]:
G_p = 1/(s_**2 + 2*zt_*s_ + 1)
G_c = K_p + K_i/s_ + K_d*s_

In [9]:
G_p

1/(s**2 + 2*s*ζ + 1)

In [10]:
T_p = feedback(G_c*G_p, 1).simplify()
T_p

(K_i + s*(K_d*s + K_p))/(K_i + s*(K_d*s + K_p) + s*(s**2 + 2*s*ζ + 1))